In [ ]:
{"table1":"""select metrics_date::date as time, sum(cast(device_retention_rate_new_daily->>'return_count' as DOUBLE PRECISION)) / sum(cast(device_retention_rate_new_daily->>'dau' as DOUBLE PRECISION)) as "Retention D7 R1"
from sapphire.sapphire_retention_master
where $__timeFilter(metrics_date)
and offset_day = 7
and application_id in (${Applications})
and market = '#Overall#'
and clientversion = '#Overall#'
and install_source in ('Organic', 'FirstParty', 'RewardsUpsell', 'PaidAds', 'Google Ads ACI', 'Upsell')
and osversion = '#Overall#'
and build = '#Overall#'
and mini_app = ''
and launch_source = '#Overall#'
and device_model = '#Overall#'
group by metrics_date""","Dau 1d-by app":"""SELECT
    '#Overall#' as "App", 
    1 as "Rolling",
    Overall as "DAU", 
    Overall-LastOverall as "WoW -7d", (Overall-LastOverall)/LastOverall as "WoW% -7d", 
    Overall-Last2_Overall as "WoW -14d", (Overall-Last2_Overall)/Last2_Overall as "WoW% -14d"
FROM (
    SELECT  
        sum(case when metrics_date = last_date then Overall else 0 end) as Overall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-7 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as LastOverall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-14 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as Last2_Overall
    FROM (
        SELECT 
            metrics_date,
            CASE WHEN cast(${__to:date:'YYYY-MM-DD'} as TEXT)>='${DauMetricsDate}' THEN cast('${DauMetricsDate}' as TEXT) ELSE cast(${__to:date:'YYYY-MM-DD'} as TEXT) END as last_date,
            SUM(CAST(metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION)) as Overall
        FROM sapphire.sapphire_engagement_metrics_master
        WHERE metrics_date>= to_char(cast(cast(${__from:date:'YYYY-MM-DD'} as timestamp) + '- 14 DAY' as date), 'yyyy-mm-dd')::TEXT AND metrics_date <= ${__to:date:'YYYY-MM-DD'}
              AND application_id in ('2130688B018F4B44BBED68E7A42BBA1E','AE427635ADC245AE973038BCB3D7C21B','4DC5714ABCAD449BA13A9B701A3CF296','4A5B528B59954AAE8725B509A41FBF1A','F185A93DE6764B098D55089F610A3FB8','FC320C411FC12CD4DFBE9A00F3161364')
              AND period = 1
              AND market = '#Overall#'
              AND os_version = '#Overall#'
              AND device_model = '#Overall#'
              AND client_version = '#Overall#'
              AND client_build_type = '#Overall#'
              AND install_channel_l1 = '#Overall#'
              AND install_channel_l2 = '#Overall#'
              AND install_channel_l3 = '#Overall#'
              AND install_channel_l4 = '#Overall#'
              AND mini_app = ''
              AND first_launch_source = '#Overall#'
              AND launch_source = '#Overall#'
        GROUP BY 
            metrics_date
    ) as a
) as b"""
}